1 ЗАДАНИЕ

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Загрузка данных
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Нормализация данных
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Преобразование меток в one-hot encoding
y_train_categorical = keras.utils.to_categorical(y_train, 10)
y_test_categorical = keras.utils.to_categorical(y_test, 10)

# Базовая модель
def create_base_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Модель с измененной архитектурой (добавлен слой, изменен размер фильтра)
def create_modified_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)),  # Изменен размер фильтра
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (5, 5), activation='relu'),  # Изменен размер фильтра
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(128, (3, 3), activation='relu'),  # Добавлен новый слой
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Обучение и оценка моделей
def train_and_evaluate(model, model_name):
    print(f"\n=== Обучение {model_name} ===")
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(x_train, y_train_categorical,
                       epochs=10,
                       batch_size=64,
                       validation_data=(x_test, y_test_categorical),
                       verbose=1)

    test_loss, test_acc = model.evaluate(x_test, y_test_categorical, verbose=0)
    print(f"{model_name} - Точность на тесте: {test_acc:.4f}")

    return history, test_acc

# Сравнение моделей
base_model = create_base_model()
modified_model = create_modified_model()

base_history, base_acc = train_and_evaluate(base_model, "Базовая модель")
modified_history, modified_acc = train_and_evaluate(modified_model, "Модифицированная модель")

print(f"\n=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
print(f"Базовая модель точность: {base_acc:.4f}")
print(f"Модифицированная модель точность: {modified_acc:.4f}")
print(f"Разница: {modified_acc - base_acc:.4f}")

2 ЗАДАНИЕ

In [ ]:
def compare_optimizers_and_epochs():
    optimizers = {
        'adam': keras.optimizers.Adam(learning_rate=0.001),
        'sgd': keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
        'rmsprop': keras.optimizers.RMSprop(learning_rate=0.001)
    }

    epochs_list = [10, 20]

    results = {}

    for opt_name, optimizer in optimizers.items():
        for epochs in epochs_list:
            print(f"\n=== {opt_name.upper()} - {epochs} эпох ===")

            model = create_base_model()
            model.compile(optimizer=optimizer,
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

            history = model.fit(x_train, y_train_categorical,
                              epochs=epochs,
                              batch_size=64,
                              validation_data=(x_test, y_test_categorical),
                              verbose=0)

            test_loss, test_acc = model.evaluate(x_test, y_test_categorical, verbose=0)
            results[f"{opt_name}_{epochs}epochs"] = {
                'accuracy': test_acc,
                'history': history
            }

            print(f"Точность: {test_acc:.4f}")

    return results

# Запуск сравнения
results = compare_optimizers_and_epochs()

# Визуализация результатов
plt.figure(figsize=(15, 5))

# График точности
plt.subplot(1, 2, 1)
for key, result in results.items():
    plt.plot(result['history'].history['val_accuracy'], label=key)
plt.title('Точность на валидации')
plt.xlabel('Эпохи')
plt.ylabel('Точность')
plt.legend()

# График потерь
plt.subplot(1, 2, 2)
for key, result in results.items():
    plt.plot(result['history'].history['val_loss'], label=key)
plt.title('Потери на валидации')
plt.xlabel('Эпохи')
plt.ylabel('Потери')
plt.legend()

plt.tight_layout()
plt.show()

# Анализ результатов
print("\n=== АНАЛИЗ РЕЗУЛЬТАТОВ ===")
for key, result in results.items():
    print(f"{key}: {result['accuracy']:.4f}")


=== ADAM - 10 эпох ===


3 ЗАДАНИЕ

In [ ]:
def analyze_errors_and_confusion_matrix(model, x_test, y_test):
    # Предсказания модели
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = y_test.flatten()

    # Confusion Matrix
    cm = confusion_matrix(y_true_classes, y_pred_classes)

    # Визуализация Confusion Matrix
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Предсказанные классы')
    plt.ylabel('Истинные классы')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # Анализ наиболее часто путающихся классов
    print("\n=== АНАЛИЗ ОШИБОК ===")

    # Нормированная confusion matrix для анализа процентов
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Находим пары классов, которые чаще всего путаются
    confusion_pairs = []
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            if i != j and cm[i, j] > 0:
                confusion_pairs.append((i, j, cm[i, j], cm_normalized[i, j]))

    # Сортируем по количеству ошибок
    confusion_pairs.sort(key=lambda x: x[2], reverse=True)

    print("Наиболее часто путающиеся классы:")
    for i, j, count, percentage in confusion_pairs[:10]:
        print(f"{class_names[i]} -> {class_names[j]}: {count} ошибок ({percentage:.2%})")

    # Визуализация примеров ошибок
    visualize_error_examples(model, x_test, y_true_classes, class_names)

def visualize_error_examples(model, x_test, y_true, class_names):
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Находим индексы ошибок
    errors = np.where(y_pred_classes != y_true)[0]

    # Выбираем случайные примеры ошибок
    np.random.shuffle(errors)
    error_indices = errors[:12]

    # Визуализация
    plt.figure(figsize=(15, 10))
    for i, idx in enumerate(error_indices):
        plt.subplot(3, 4, i + 1)
        plt.imshow(x_test[idx])
        plt.title(f'Истинный: {class_names[y_true[idx]]}\nПредсказанный: {class_names[y_pred_classes[idx]]}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Используем лучшую модель для анализа ошибок
best_model = create_base_model()
best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
best_model.fit(x_train, y_train_categorical, epochs=10, batch_size=64, verbose=0)

# Анализ ошибок
analyze_errors_and_confusion_matrix(best_model, x_test, y_test)